In [ ]:
import xgi                                   # HIGH‑ORDER graphs in Python   [oai_citation:0‡xgi.readthedocs.io](https://xgi.readthedocs.io/?utm_source=chatgpt.com)

def build_KB():
    H = xgi.DiHypergraph()
    H.add_nodes_from(range(11))              # 0 … 10

    # unary   n → n+1
    for n in range(9):
        print(n, n+1)
        H.add_edge(([n], [n+1]), op="inc",  id=f"inc_{n}")

    # binary  {a,b} → a+b   (only while sum ≤ 10 to stay inside universe)
    for a in range(11):
        for b in range(11 - a):
            H.add_edge([a, b], [a + b],
                       op="add",
                       id=f"add_{a}_{b}_{a+b}")
    return H

KB = build_KB()         # immutable knowledge


0 1


IndexError: list index out of range

In [3]:
def inject_problem(H, a, b):
    """Add control nodes and tell the graph we want a+b."""
    H.add_nodes_from(["initial", "final", "memory"])

    # single hyperedge: tail = {initial}, heads = {a,b}
    H.add_edge(["initial"], [a, b],
               op="want", id=f"want_{a}_{b}")

In [4]:
def one_rewrite_step(H) -> bool:
    """
    If there is a 'want' hyperedge (initial ▷ a,b) **and**
    an 'add' hyperedge (a,b ▷ c),   create result edge
    (final ▷ c) and store (memory ▷ c).  Remove the 'want'.
    Returns True if something changed.
    """
    for w in list(H.edges):
        if H.edges[w].get("op") != "want":
            continue

        tails = list(H.edges[w].tail)        # = ['initial']
        a, b = H.edges[w].head               # two operands

        # look up the unique add‑edge {a,b}→c
        for e in H.nodes[a].edges & H.nodes[b].edges:
            if H.edges[e].get("op") == "add":
                c = next(iter(H.edges[e].head))   # the sum node

                # write the answer
                H.add_edge(["final"],   [c], op="result",
                           id=f"res_{a}_{b}_{c}")
                H.add_edge(["memory"],  [c], op="stored",
                           id=f"mem_{a}_{b}_{c}")

                # clean up
                H.remove_edge(w)
                return True                 # fired!
    return False                            # nothing to do

In [5]:
def run_until_halt(H, max_steps=10):
    for t in range(max_steps):
        if any(H.edges[e].get("op") == "result" for e in H.edges):
            return t                        # halted
        if not one_rewrite_step(H):
            raise RuntimeError("Stuck – no rule applicable")
    raise RuntimeError("Did not halt in max_steps")

In [ ]:
inject_problem(KB, 2, 3)

    steps = run_until_halt(KB)
    res_edge = next(e for e in KB.edges if KB.edges[e].get("op") == "result")
    result  = next(iter(KB.edges[res_edge].head))

    print(f"Halted in {steps} step(s);   2 + 3 = {result}")